In [ ]:
lulc_rst = '/mnt/d/autocls01/fidx/lulc_ref.tif'

idx_rst = {
    "20220504" : {
        "ndvi" : '/mnt/d/autocls01/fidx/ndvi_20220504.tif',
        "ndbi" : '/mnt/d/autocls01/fidx/ndbi_20220504.tif',
        "ndwi" : '/mnt/d/autocls01/fidx/ndwi_20220504.tif'
    },
    "20220812" : {
        "ndvi" : '/mnt/d/autocls01/fidx/ndvi_20220812.tif',
        "ndbi" : '/mnt/d/autocls01/fidx/ndbi_20220812.tif',
        "ndwi" : '/mnt/d/autocls01/fidx/ndwi_20220812.tif'
    },
    "20221001" : {
        "ndvi" : '/mnt/d/autocls01/fidx/ndvi_20221001.tif',
        "ndbi" : '/mnt/d/autocls01/fidx/ndbi_20221001.tif',
        "ndwi" : '/mnt/d/autocls01/fidx/ndwi_20221001.tif'
    }
}

idx_rules = {
    1 : {
        "ndbi" : {'operator' : '>', 'value': 0, },
        "ndvi" : {'operator' : '<', 'value': 0.3},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    2 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    3 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    4 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    5 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0.3},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    6 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    7 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '>', 'value': 0},
        "ndwi" : {'operator' : '<', 'value': 0}
    },
    8 : {
        "ndbi" : None,
        "ndvi" : {'operator' : '<', 'value': 0.3},
        "ndwi" : {'operator' : '>', 'value': 0}
    }
}

fraster= '/mnt/d/autocls01/fidx/lulc_filter.tif'

In [ ]:
import os
from osgeo import gdal
import numpy as np
from glass.rd.rst import rst_to_array
from glass.prop.rst import get_nodata
from glass.pys.oss import lst_ff, fprop
from glass.wenv.grs import run_grass
from glass.pys.tm import now_as_str

In [ ]:
# Check if outfolder exists
out_folder = os.path.dirname(fraster)
if not os.path.exists(out_folder):
    mkdir(out_folder, overwrite=None)
    
"""
Start GRASS GIS Session
"""

loc = f'loc_{now_as_str()}'
grsb = run_grass(
    out_folder, grassBIN='grass78', location=loc,
    srs=lulc_rst
    )

import grass.script.setup as gsetup
    
gsetup.init(grsb, out_folder, loc, 'PERMANENT')

In [ ]:
from glass.it.rst import rst_to_grs, grs_to_rst
from grass.pygrass.modules import Module

In [ ]:
# Identificar classes existentes no raster com as classes de LULC

lulc_img = rst_to_array(lulc_rst)

ndval = get_nodata(lulc_rst)

lulc_class = [v for v in np.unique(lulc_img) if v != ndval]

In [ ]:
print(lulc_class)

In [ ]:
# Send all rasters to GRASS GIS

lulc_grs = rst_to_grs(lulc_rst, fprop(lulc_rst, 'fn'))

for d in idx_rst:
    for k in idx_rst[d]:
        idx_rst[d][k] = rst_to_grs(idx_rst[d][k], fprop(idx_rst[d][k], 'fn'))

In [ ]:
idx_rst

Para cada índice de uma determinada data, criar um ficheiro que indica a classe dos pixeis a incluir no treino (basta ter em conta o valor do índice)

In [ ]:
mask_by_day

In [ ]:
mask_by_day = {}
for d in idx_rst:
    idxd = {}
    for k in idx_rst[d]:
        ifs = []
        
        for c in range(len(lulc_class)):
            _i = len(lulc_class) - 1 - c
            
            if not idx_rules[lulc_class[_i]][k]: continue
            
            form = (
                f"if({lulc_grs} == {str(lulc_class[_i])} && "
                f"{idx_rst[d][k]} {idx_rules[lulc_class[_i]][k]['operator']} "
                f"{idx_rules[lulc_class[_i]][k]['value']}, {str(lulc_class[_i])}, "
                f"{'0' if not len(ifs) else ifs[c-1]})"
            )
            ifs.append(form)
        
        outmask = f'mask_{k}_{d}'
        rc = Module ('r.mapcalc', expression=f'{outmask} = {ifs[-1]}')
        
        idxd[k] = outmask
    
    mask_by_day[d] = idxd

In [ ]:
for d in mask_by_day:
    for k in mask_by_day[d]:
        grs_to_rst(mask_by_day[d][k], os.path.join(
            os.path.dirname(fraster),
            f'{mask_by_day[d][k]}.tif'
        ), is_int=True)

Para cada índice, criar um ficheiro que indica a classe dos pixeis a incluir no treino, tendo em conta as máscaras obtidas em cada data considerada

Obter máscara final a partir das máscaras produzidas para cada índice